In [54]:
import joblib
import numpy as np
import uvicorn
import mlflow
import mdsist.util as util

from fastapi import FastAPI
from pydantic import BaseModel

# Load packages
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from mdsist.dataset import MdsistDataset
from mdsist.config import PROCESSED_DATA_DIR
from mdsist.predictor import Predictor

import asyncio

import mlflow
import mlflow.pytorch

import mdsist.util as util

In [42]:
from dotenv import load_dotenv

# Load envionment variables
load_dotenv()

True

In [46]:
## Import model from MLFlow, with below URI
MODEL_URI = 'runs:/10cb51b288134c48835a8c0b9fe66eca/model_20240930190709'
#'models:/CNNv1-production/'

device = util.get_available_device()

model = mlflow.pytorch.load_model(MODEL_URI, map_location=device)

pred = Predictor(model)

In [48]:
app = FastAPI()

In [50]:
class PredictionRequest(BaseModel):
    image: list # The input data should be an array list of 256 pixel

@app.post("/mnist-model-prediction/")
async def predict(data: PredictionRequest):
    # pass the image as byte, then from buffer.
    
    image_array = np.frombuffer(dtype= np.uint8) # this method interprets a buffer as a 1D array. 

    # reshape uint8 to below structure
    # shape (N, 1, H, W) # N = degree of freedom, -1.  H = 28, W = 28

    images = image_array.reshape((-1, 1, 28, 28))
    
    prediction = pred.predict(images)

    print(prediction)
    #return{"prediction": int(prediction[0])}


In [56]:
async def main():
    config = uvicorn.Config("app:app")
    server = uvicorn.Server(config)
    await server.serve()

In [58]:
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [52]:
uvicorn.run("app:app")

RuntimeError: asyncio.run() cannot be called from a running event loop